In [7]:
%reset

Once deleted, variables cannot be recovered. Proceed (y/[n])? y


In [2]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:95% !important; }</style>"))
display(HTML("<style>.input_area pre {font-family: Consolas; font-size: 16pt; line-height: 140%;}</style>"))
display(HTML("<style>.output_area pre {font-family: Consolas; font-size: 16pt; line-height: 140%;}</style>"))

from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

from sklearn import linear_model, datasets
import sklearn.metrics as metric
from sklearn.model_selection import KFold
import pandas as pd
import numpy as np
import warnings
import os
import glob
import datetime
import shutil
from collections import defaultdict, Counter
import statsmodels.stats.multitest as multest
from copy import deepcopy
import networkx as nx
import matplotlib.pyplot as plt
from matplotlib import font_manager, rc
import scipy.stats
font_name = font_manager.FontProperties(fname="c:/Windows/Fonts/malgun.ttf").get_name()
plt.rc('font', family=font_name)
from collections import Counter
import sklearn.metrics

In [3]:
def get_sigTF_ttest(file_names):
    TF_score_integrated_df = pd.DataFrame()
    for file_name in file_names:
        cell=file_name.split("\\")[-1].split('_')[2]
        print('#####',cell)
        
        anal_data=pd.read_table(file_name, sep='\t',engine='python')
        therp_TF_score={}
        for col in anal_data.columns:
            if col=='effect':
                continue

            pos_set=anal_data.loc[anal_data['effect']==1,col]
            neg_set=anal_data.loc[anal_data['effect']==0,col]
            
            TTS, p_val=scipy.stats.ttest_ind(pos_set, neg_set, equal_var=False)

            therp_TF_score[col]= [pos_set.mean(), neg_set.mean(), TTS, p_val]

        TF_score_df=pd.DataFrame(therp_TF_score)
        TF_score_df=TF_score_df.T
        TF_score_df.columns=['TFAs Ave.(E)','TFAs Ave.(NE)', 'TTS','p_val'] ## TTS strong positive:TSG, strong negative:EG
        TF_score_df=TF_score_df.sort_values(by='TTS', ascending=False)
        
        TF_score_df['FDR'] = multest.fdrcorrection(TF_score_df['p_val'])[1]
        TF_score_df['negLog10_FDR'] = TF_score_df['FDR'].map(lambda x: -np.log10(x))
        
        ## save TF therapeutics score for each cell
        TF_score_df.to_csv('result/TTscore/TTS_{}.txt'.format(cell),sep='\t')
        print(len(TF_score_df))

        ## integration nine cells
        for col in ['TFAs Ave.(E)','TFAs Ave.(NE)','TTS','negLog10_FDR']:
            TF_score_df[col] = TF_score_df[col].map(lambda x: '{:.3f}'.format(x))
        
        for col in ['p_val','FDR']:
            TF_score_df[col] = TF_score_df[col].map(lambda x: '{:.3e}'.format(x))
        
        TF_score_df.rename(columns={'FDR':'FDR_{}'.format(cell)}, inplace=True)
        
        TF_score_integrated_df=pd.concat([TF_score_integrated_df, TF_score_df], axis=1, sort=True)
        
    TF_score_integrated_df.to_csv('result/TTscore/TTS_both.txt',sep='\t')
    
file_names=glob.glob('result/TF_activity/effect/*.txt')  
get_sigTF_ttest(file_names)

##### A375
627
##### HT29
627
